In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import sys
# 将 JumpGP_code_py 所在的目录添加到 Python 路径
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
# sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import torch
import numpy as np
import math

from utils1 import jumpgp_ld_wrapper

from VI_utils_gpu_accelerate import *
from JumpGP_test import *

Using device: cuda


d:\new_windows\envs\torch-py310-yxu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
folder_name = "2025_04_02_01_21"
dataset = load_dataset(folder_name)
X_train = dataset["X_train"]
Y_train = dataset["Y_train"]
X_test = dataset["X_test"]
Y_test = dataset["Y_test"]

neighborhoods = find_neighborhoods(X_test, X_train, Y_train, M=200)

In [9]:
neighborhoods[0].keys()

dict_keys(['X_neighbors', 'y_neighbors', 'indices'])

In [12]:
neighborhoods[0]['X_neighbors'].shape, neighborhoods[0]['y_neighbors'].shape

(torch.Size([200, 15]), torch.Size([200]))

In [15]:
X_test.shape, Y_test.shape

(torch.Size([500, 15]), torch.Size([500]))

In [2]:
import torch

# 假设你之前已经有这些
folder_name = "2025_04_02_01_21"
dataset = load_dataset(folder_name)
X_train = dataset["X_train"]  # shape: (N_train, D)
Y_train = dataset["Y_train"]
X_test = dataset["X_test"]    # shape: (N_test, D)
Y_test = dataset["Y_test"]

# 自动推导
N_test, D = X_test.shape
N_train = X_train.shape[0]
Q = 2  # 自定义潜在维度，比如你想设成2
m1 = 3  # 每个region的 inducing points数量
m2 = 4  # 全局 inducing points数量
T = N_test  # 每个测试点对应一个region（你可以根据需要调整，比如分块）
n = 200  # 每个region的邻居数量（因为你 find_neighborhoods 时设了 M=200）

# 使用 neighborhoods 建立 regions
neighborhoods = find_neighborhoods(X_test, X_train, Y_train, M=n)

regions = []
for i in range(T):
    regions.append({
        'X': neighborhoods[i]['X_neighbors'],  # shape (n, D)
        'y': neighborhoods[i]['y_neighbors'],  # shape (n,)
        'U': 1.0,  # 可以固定为1.0，后续优化
        'C': torch.randn(m1, Q)  # 随机初始化 C
    })

# 初始化 V_params
V_params = {
    'mu_V': torch.randn(m2, Q, D, requires_grad=True),
    'sigma_V': torch.rand(m2, Q, D, requires_grad=True)
}

# 初始化 u_params
u_params = []
for _ in range(T):
    u_params.append({
        'mu_u': torch.randn(m1, requires_grad=True),
        'Sigma_u': torch.eye(m1, requires_grad=True),
        'sigma_noise': torch.tensor(0.5, requires_grad=True),
        'omega': torch.randn(Q+1, requires_grad=True)
    })

# 初始化 hyperparams，特别是 Z 用 X_train 分布初始化
X_train_mean = X_train.mean(dim=0)
X_train_std = X_train.std(dim=0)
Z = X_train_mean + torch.randn(m2, D) * X_train_std

hyperparams = {
    'Z': Z,  # m2 x D，模拟X_train分布
    'X_test': X_test,  # 全部T个测试点
    'lengthscales': torch.rand(Q, requires_grad=True),
    'var_w': torch.tensor(1.0, requires_grad=True),
    # 'sigma_f' 可以自己加
}

# （下面是继续用你之前的流程）
L = compute_ELBO(regions, V_params, u_params, hyperparams)
print("ELBO L =", L.item())
L.backward()
print("Gradients OK")

V_params, u_params, hyperparams = train_vi(
    regions=regions,
    V_params=V_params,
    u_params=u_params,
    hyperparams=hyperparams,
    lr=1e-3,
    num_steps=10,
    log_interval=10
)
print("train OK")

mu_pred, var_pred = predict_vi(regions, V_params, hyperparams, M=10)
print("Prediction OK")


ELBO L = -207241.8125
Gradients OK
Step 1/10, ELBO = -207241.8125


KeyboardInterrupt: 

In [3]:
import torch

# 1. 设备选择：优先使用 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 加载并搬到 device
folder_name = "2025_04_02_01_21"
dataset = load_dataset(folder_name)
X_train = dataset["X_train"].to(device)   # (N_train, D)
Y_train = dataset["Y_train"].to(device)
X_test  = dataset["X_test"].to(device)    # (N_test, D)
Y_test  = dataset["Y_test"].to(device)

# 3. 自动推导维度
N_test, D = X_test.shape
N_train   = X_train.shape[0]
Q   = 2     # 潜在维度
m1  = 3     # 每个 region 的 inducing points 数量
m2  = 4     # 全局 inducing points 数量
T   = N_test
n   = 200   # 每个 region 的邻居数量

# 4. 构造 neighborhoods，并把它们搬到 GPU
neighborhoods = find_neighborhoods(X_test.cpu(), X_train.cpu(), Y_train.cpu(), M=n)
# （find_neighborhoods 内部可能要求 CPU 张量，所以我们传入 cpu()，然后再搬回 GPU）
regions = []
for i in range(T):
    X_nb = neighborhoods[i]['X_neighbors'].to(device)  # (n, D)
    y_nb = neighborhoods[i]['y_neighbors'].to(device)  # (n,)
    regions.append({
        'X': X_nb,
        'y': y_nb,
        'U': 1.0,                        # 常数
        'C': torch.randn(m1, Q, device=device)  # 随机初始化
    })

# 5. 初始化 V_params（放到 GPU）
V_params = {
    'mu_V':    torch.randn(m2, Q, D, device=device, requires_grad=True),
    'sigma_V': torch.rand( m2, Q, D, device=device, requires_grad=True),
}

# 6. 初始化 u_params（放到 GPU）
u_params = []
for _ in range(T):
    u_params.append({
        'mu_u':         torch.randn(m1, device=device, requires_grad=True),
        'Sigma_u':      torch.eye(m1, device=device, requires_grad=True),
        'sigma_noise':  torch.tensor(0.5, device=device, requires_grad=True),
        'omega':        torch.randn(Q+1, device=device, requires_grad=True),
    })

# 7. 初始化 hyperparams（放到 GPU）
#    用 X_train 的分布来初始化 Z
X_train_mean = X_train.mean(dim=0)
X_train_std  = X_train.std(dim=0)
Z = X_train_mean + torch.randn(m2, D, device=device) * X_train_std

hyperparams = {
    'Z':             Z,                     # (m2, D)
    'X_test':        X_test,                # (T, D)
    'lengthscales':  torch.rand(Q, device=device, requires_grad=True),
    'var_w':         torch.tensor(1.0, device=device, requires_grad=True),
}

print("Everything set!")

# 8. 计算 ELBO、反向传播、训练、预测（所有计算都在 GPU 上）
L = compute_ELBO(regions, V_params, u_params, hyperparams)
print("ELBO L =", L.item())
L.backward()
print("Gradients OK")

V_params, u_params, hyperparams = train_vi(
    regions=regions,
    V_params=V_params,
    u_params=u_params,
    hyperparams=hyperparams,
    lr=1e-3,
    num_steps=10,
    log_interval=10
)
print("train OK")

mu_pred, var_pred = predict_vi(
    regions,
    V_params,
    hyperparams,
    M=10
)
print("Prediction OK")
print("mu_pred:", mu_pred)
print("var_pred:", var_pred)


Using device: cuda
Everything set!
ELBO L = -195001.96714135353
Gradients OK
Step 1/10, ELBO=-195001.9671
Step 10/10, ELBO=-192615.2382
train OK
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 2
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 2
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 1
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iteration 0
maximize_PD func, we fail at iterat

In [4]:
rmse, q25, q50, q75 = compute_metrics(mu_pred, var_pred, Y_test)
rmse, q25, q50, q75

(tensor(2.2795, device='cuda:0'),
 tensor(-0.3767, device='cuda:0'),
 tensor(4.6729, device='cuda:0'),
 tensor(41.7138, device='cuda:0'))

In [6]:
mu_pred

tensor([4.9597, 5.0085, 4.9843, 5.0036, 5.0434, 5.0147, 5.0102, 5.0027, 4.9975,
        5.0065, 5.0060, 5.0259, 5.1056, 4.9976, 4.9916, 4.9339, 4.9994, 5.0114,
        4.9204, 4.9970, 5.0039, 4.9967, 5.0000, 4.9651, 5.0066, 5.0060, 4.9956,
        5.0049, 5.0165, 5.0117, 5.0026, 5.0062, 4.9977, 5.0080, 5.0016, 5.0102,
        4.9857, 5.0174, 4.9950, 5.0081, 5.0254, 5.0115, 4.9980, 5.0056, 5.0101,
        4.9990, 4.9958, 4.9671, 4.9954, 5.0127, 5.0210, 5.0170, 5.0052, 4.9972,
        5.0031, 5.0016, 4.9925, 4.9835, 4.9872, 5.0029, 5.0155, 4.7062, 5.0112,
        5.0176, 5.0083, 4.9987, 4.9878, 5.0039, 5.0090, 4.9942, 5.0175, 4.9994,
        5.0159, 5.0036, 4.9870, 5.0575, 4.9875, 5.0031, 5.0093, 4.9806, 4.9983,
        5.0930, 4.9922, 5.2987, 4.9453, 5.0076, 4.9988, 4.9911, 4.9561, 4.9948,
        5.0091, 5.0105, 4.9956, 5.0119, 5.0020, 4.6307, 5.0012, 5.0092, 5.0316,
        5.0135, 4.9966, 4.9833, 5.0047, 4.8279, 5.0045, 5.0036, 5.0262, 5.0040,
        5.0044, 4.9053, 5.0141, 5.0024, 